In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import importlib
importlib.reload(htm)


<module 'HandTrackingModule' from '/Users/my/Desktop/volumeControlGesture/HandTrackingModule.py'>

In [18]:
import cv2
import time
import numpy as np
import math
import HandTrackingModule as htm

In [24]:
wCam, hCam = 640, 480

In [7]:
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

In [21]:
detector = htm.handDetector()
